# Instruction Power Differences

This tutorial will introduce you to measuring the power consumption of a device under attack. It will demonstrate how the power consumption of a target changes based on what operations it's doing.

If you haven't yet, you should probably complete Tutorial B1, which introduces building firmware, programming the target, and scripting.

## Setting up Firmware

In this tutorial, we will once again be working off of the simpleserial-base firmware.

Let's start by creating a new project and building our firmware:

In [ ]:
%%bash
cd ../../hardware/victims/firmware/
mkdir -p simpleserial-base-lab2 && cp -r simpleserial-base/* $_
cd simpleserial-base-lab2

In [ ]:
PLATFORM = "CWLITEARM"
CRYPTO_TARGET = "NONE"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

As in the previous tutorial, we'll need to modify our firmware. Navigate to the `get_pt()` function:
```C
uint8_t get_pt(uint8_t* pt)
{
	/**********************************
	* Start user-specific code here. */
	trigger_high();
	
	//16 hex bytes held in 'pt' were sent
	//from the computer. Store your response
	//back into 'pt', which will send 16 bytes
	//back to computer. Can ignore of course if
	//not needed
	
	trigger_low();
	/* End user-specific code here. *
	********************************/
	simpleserial_put('r', 16, pt);
	return 0x00;
}

```

To start off, we'll add a simple `for` loop. We'll start off by looking at how the power trace changes based on how long the loop is. Start with a loop (make sure your variables are volatile) that runs from 0 to 4:

```C
for(volatile int i = 0; i < 5; i++);
```

Next, we'll move on to actually capturing and displaying the power trace.

## ChipWhisperer Setup

Setup for this tutorial will be pretty similar to Tutorial B1, so we'll skip most of it by calling some helper scripts. This setup should work for most targets, but if you're using a target other than the XMEGA or STM32F3 (CWLite w/ Arm), you may need to call a different script or do additional setup (like programming the target with an external programmer). See the wiki page for your target for more information.

If you're curious about what's happening in these helper scripts, they're typically located in the `Helper_Scripts` folder.

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
import chipwhisperer as cw
scope = cw.scope()
target = cw.target(scope)

In [ ]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

By default, the scope will capture many more traces than we need, so we'll reduce that to 1000.

In [ ]:
scope.adc.samples = 1000

Next we'll get the function we'll use to program the target. These scripts define `program_target(scope, firmware)`, which we'll use in the next block.  We'll also set the `fw_path` variable, which will save us from having to reenter the long path to our firmware every time we want to program. Make sure this path is correct.

In [ ]:
# uncomment based on your target
fw_path = "../../hardware/victims/firmware/simpleserial-base-lab2/simpleserial-base-CWLITEARM.hex"
%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/No_Programmer.ipynb"

In [ ]:
program_target(scope, fw_path)

## Capturing Traces

Like before, most of this should look familiar from the last tutorial. We'll start by programming the target, then capturing a trace, and finally displaying it using bokeh. We don't really care about what the target responds with this time, so we won't do anything with what we read back.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
output_notebook()

def get_trace():
    ktp = cw.ktp.Basic(target=target)
    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here

    target.loadInput(text)
    scope.arm()
    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)
    ret = scope.capture() #capture trace
    trace = scope.getLastTrace() #get trace
    _ = target.readOutput()  # clears the response from the serial port
    return trace

    
    
trace = get_trace()
xrange = range(len(trace))
p = figure()
p.line(xrange, trace, line_color="red")
show(p)

## Further Modifications

You might be able to tell where our loop is happening, but to make sure, let's modify it and see how the trace changes. Change when the loop ends and save the file. Rebuild the file:

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

Reprogram the target:

In [ ]:
program_target(scope, fw_path)

And capture a new trace:

In [ ]:
trace2 = get_trace()
xrange = range(len(trace2))
p = figure()
p.line(xrange, trace2, line_color="red")
show(p)

It should now be very obvious where our loop is happening. Verify that the number of similar looking power spikes matches how many times the loop is run. Now let's take a look at how different operations affect power traces:

## Comparing Operations

To make things easy on us, let's select operations that we would expect to have very different power traces. One easy metric to base your decision on is how long the associated instructions take to execute (which is typically documented either in the datasheet, or in the core's documentation). For example, on XMEGA we'll compare adding (1 cycle) with multiplying (2 cycles).

### XMEGA

### STM32F3

Our reference for the STM32F3 (which has an Arm Cortex M4 CPU) will the [Cortex M4 Instruction Set Summary](http://infocenter.arm.com/help/index.jsp?topic=/com.arm.doc.ddi0439b/CHDDIGAC.html). As we can see, the multiply instruction takes 1 cycle to execute, while the divide instruction takes between 2 and 12. These instructions also have analogous C operations (* and /) so it will be easy to get the compiler to insert them.

This is a pretty drastic difference, so the two instructions should look very different. Change your loop so that it preforms multiplications each time through:

```C
volatile int b = 0xAFFA;
for (volatile int i = 0; i < 10; i++)
    b *= 11;
```

### Capture: Operation 1

Now rebuild and capture another trace:

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
program_target(scope, fw_path)

In [ ]:
trace3 = get_trace()
xrange = range(len(trace3))
p = figure()
p.line(xrange, trace3, line_color="red")
show(p)

### Capture: Operation 2

Replace your first operation with your second. Save the file, rebuild, program, and capture another trace.

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
program_target(scope, fw_path)

In [ ]:
trace4 = get_trace()
xrange = range(len(trace4))
p = figure()
p.line(xrange, trace4, line_color="red")
show(p)

Compare the traces. Is the one that you expected to be longer actually longer?

The longer operation is typically more complicated than the other operation, which means we would expect it to have higher power consumptions as well. Do you also see this behaviour in the traces you captured?

## Disconnecting from ChipWhisperer

Now that we're done with the tutorial, we'll need to disconnect from ChipWhisperer, so that we can connect to it in a different tutorial:

In [ ]:
scope.dis()
target.dis()

## Conclusion

In this tutorial you have learned how power analysis can tell you the operations being performed on a microcontroller. In future work we will move towards using this for breaking various forms of security on devices. In particular, Tutorial B3-1 will examine how we can use this information to exploit a password check.